<a href="https://colab.research.google.com/github/nrohrbach/PocDatenkatalogBafu/blob/main/PoC_Datenkatalog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Test
Daten: http://www.bafu-daten.ch/wasser/restwasser/data/data/index/rawData_d.txt

In [13]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [19]:
Options = ['Abfall', 'Altlasten','Bildung, Forschung, Innovation','Biodiversität'
           ]

In [20]:
def map_options(option):
  mapping = {
      'Abfall': 'abfall',
      'Altlasten': 'altlasten',
      'Bildung, Forschung, Innovation': 'bildung',
      'Biodiversität': 'biodiversitat'
  }
  return mapping.get(option, None) # Return None if option is not found


In [21]:
map_options('Abfall')

'abfall'

In [14]:
# Funktion fragt alle Datensätze von opendata.swiss nach einem Keyword ab

def search_datasets(keyword):
    # Basis-URL der CKAN API
    base_url = "https://opendata.swiss/api/3/action/package_search"

    # Parameter für die Abfrage
    params = {
        "fq": f"organization:{'bundesamt-fur-umwelt-bafu'}",
        "q": f"keywords_de:{keyword}"
    }

    # Anfrage an die API senden
    response = requests.get(base_url, params=params)
    data = response.json()
    data = data["result"]["results"]

    # Daten aus Json auslesen
    description = [s['description']['de'] for s in data]
    title = [s['title']['de'] for s in data]
    url0 = [s['resources'][0]['url'] for s in data]
    title0 = [s['resources'][0]['title']['de'] for s in data]
    url1 = [s['resources'][1]['url'] for s in data]
    title1 = [s['resources'][1]['title']['de'] for s in data]

    # Daten als Dataframe speichern
    dict = {'title': title, 'description': description, 'url0': url0, 'title0': title0, 'url1': url1, 'title1': title1}
    dfOpendataSwiss = pd.DataFrame(dict)


    url = "https://www.bafu.admin.ch/bafu/de/home/zustand/indikatoren.html"
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for bad status codes

    soup = BeautifulSoup(response.content, "html.parser")
    table_body = soup.find("table", attrs={"class": "table"}) # Corrected table identification

    if table_body:
        rows = table_body.find_all("tr")
        data = []
        for row in rows:
            cells = row.find_all("td")
            if len(cells) >= 2:  # Ensure there are at least two cells
                thema = cells[0].text.strip()
                indikatorname = cells[1].text.strip()
                # Extract href from the second cell (index 1)
                link_element = cells[1].find('a')
                if link_element:
                    href = link_element.get('href')
                    data.append({"Thema": thema, "Indikatorname": indikatorname, "Link": "https://www.bafu.admin.ch"+ href})
                else:
                    data.append({"Thema": thema, "Indikatorname": indikatorname, "Link": None}) # Handle cases where no link is found.


        df = pd.DataFrame(data)
    df



    return dfOpendataSwiss

In [18]:
search_datasets('luft')

,title,description,url0,title0,url1,title1


In [17]:
url = "https://www.bafu.admin.ch/bafu/de/home/zustand/indikatoren.html"
response = requests.get(url)
response.raise_for_status()  # Raise an exception for bad status codes

soup = BeautifulSoup(response.content, "html.parser")
table_body = soup.find("table", attrs={"class": "table"}) # Corrected table identification

if table_body:
    rows = table_body.find_all("tr")
    data = []
    for row in rows:
        cells = row.find_all("td")
        if len(cells) >= 2:  # Ensure there are at least two cells
            thema = cells[0].text.strip()
            indikatorname = cells[1].text.strip()
            # Extract href from the second cell (index 1)
            link_element = cells[1].find('a')
            if link_element:
                href = link_element.get('href')
                data.append({"Thema": thema, "Indikatorname": indikatorname, "Link": "https://www.bafu.admin.ch"+ href})
            else:
                data.append({"Thema": thema, "Indikatorname": indikatorname, "Link": None}) # Handle cases where no link is found.


    df = pd.DataFrame(data)
df

,Thema,Indikatorname,Link
0,Abfall,Abfallverbrennung,https://www.bafu.admin.ch/bafu/de/home/themen/...
1,Abfall,Energieeffizienz bei der Abfallverbrennung,https://www.bafu.admin.ch/bafu/de/home/themen/...
2,Abfall,Verwertete Sonderabfälle,https://www.bafu.admin.ch/bafu/de/home/themen/...
3,"Abfall, Wirtschaft u...",Recyclingquote,https://www.bafu.admin.ch/bafu/de/home/themen/...
4,"Abfall, Wirtschaft u...",Siedlungsabfälle,https://www.bafu.admin.ch/bafu/de/home/themen/...
...,...,...,...
194,Wasser,Hohe Seepegelstände,https://www.bafu.admin.ch/bafu/de/home/themen/...
195,Wasser,Wasserbilanz und Wasserspeicher,https://www.bafu.admin.ch/bafu/de/home/themen/...
196,"Wasser, Biodiversitä...",Verkäufe von Pflanzenschutzmitteln,https://www.bafu.admin.ch/bafu/de/home/themen/...
197,"Wasser, Klima",Schneemenge,https://www.bafu.admin.ch/bafu/de/home/themen/...
